# LASSO with a single "Gaussian" observation

Let $y\in\mathbb{R}, \sigma\in\mathbb{R}_+^*$ and consider 

$$
\underset{u\in L^2(\mathbb{R}^2)}{\text{inf}} ~ \frac{1}{2}\left(\int_{\mathbb{R}^2}\text{exp}(-\frac{||x||^2}{2\sigma^2}) ~ u(x) ~ dx - y\right)^2 + \lambda |Du|(\mathbb{R}^2)
$$

One can show that there exists a solution of the form $u_{\alpha,R}=a\,\mathbb{1}_{B(0, R)}$

Now one can also show that $u_{\alpha,R}$ is a solution if and only if $R$ satisfies $1+\frac{R^2}{\sigma^2}=\text{exp}(\frac{R^2}{2\sigma^2})$ and 
$a=y \, / \, [2\pi\sigma^2(1-\text{exp}(-1/2))]$

Let us investigate whether we are able to recover this result numerically or not ...

## Numerics

PyCheeger commit : 872cdb0a2c9deef14c3d99b0e6805292eba24ca6
<br/>
tvsfw commit : c42d516501c7a0ee761be79a6f7b56927a0a9f69

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from math import pi, exp
from scipy.optimize import minimize

from pycheeger import compute_cheeger, Disk
from tvsfw import GaussianPolynomial, SampledGaussianKernel, WeightedIndicatorFunction, SimpleFunction

In [2]:
sigma = 0.3
alpha = 1e-2  # regularization parameter

In [3]:
r_opt = minimize(lambda x: x / (1 - exp(-x**2 / (2 * sigma**2))), sigma).x[0]
print("theoretical optimal radius: {}".format(r_opt))

theoretical optimal radius: 0.47556030828963447


In [4]:
y = 2 * pi * sigma ** 2 * (1 - exp(-r_opt ** 2/ (2 * sigma ** 2)))  # we here define y so that ...

In [5]:
grid = np.array([[[0, 0]]])
phi = SampledGaussianKernel(grid, sigma)
eta = GaussianPolynomial(grid, np.array([[y]]), sigma)

In [6]:
E, obj_tab, grad_norm_tab = compute_cheeger(eta, max_tri_area=0.001, max_primal_dual_iter=20000, max_iter=5000, convergence_tol=1e-7,
                          plot_results=True)

TypeError: triangulate() got an unexpected keyword argument 'max_area'

In [ ]:
np.linalg.norm((E.compute_perimeter() * E.compute_weighted_area_gradient(eta) - E.compute_perimeter_gradient() * E.compute_weighted_area(eta)) / E.compute_weighted_area(eta)**2)

In [ ]:
np.sum(E.boundary_vertices, axis=0) / len(E.boundary_vertices)

In [ ]:
np.linalg.norm(E.boundary_vertices - np.sum(E.boundary_vertices, axis=0) / len(E.boundary_vertices), axis=1)

In [ ]:
np.linalg.norm(E.boundary_vertices, axis=1)

In [ ]:
np.linalg.norm(E.boundary_vertices[1:] - E.boundary_vertices[:-1], axis=1)

In [ ]:
u_hat = SimpleFunction([WeightedIndicatorFunction(0, E)])

No need to perform a LASSO to get the weight, it is entirely determined (single observation)

In [ ]:
def soft_thresh(x, lala):
    return np.sign(x) * np.max(x - lala, 0)

In [ ]:
obs = u_hat.compute_obs(phi, version=1)
coeff = np.sum(obs[0], axis=0)[0, 0]
weight = soft_thresh(y / coeff, alpha / coeff ** 2)
u_hat.atoms = [WeightedIndicatorFunction(weight, E)]

After adding a new atom to the support and performing a LASSO on the weights, we get the weighted indicator function of a disk with weight and radius given above.

<br/>

We can check that if we denote this function $\hat{u}$ we have 
$$\int_{\mathbb{R}^2}\hat{u}~\text{exp}(-\frac{||x||^2}{2\sigma^2})~dx=y$$


In [ ]:
radius = np.mean(np.linalg.norm(u_hat.atoms[0].support.boundary_vertices, axis=1))

print("radius: {}".format(radius))
print("weight: {}".format(weight))

In [ ]:
y / coeff - alpha / coeff ** 2

The estimated radius moreover approximately satisfies the optimality equation given at the beginning

In [ ]:
print(1 + radius ** 2 / sigma ** 2)
print(exp(radius ** 2 / (2 * sigma ** 2)))

In [ ]:
1 - alpha * radius / (sigma ** 2 * y * (1 - exp(-radius ** 2 / (2 * sigma ** 2))))

In [ ]:
obj_tab, grad_norm_tab = u_hat.perform_sliding(y, phi, alpha, 0.1, 500, 1e-7)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(obj_tab);

In [ ]:
plt.plot(grad_norm_tab);

In [ ]:
radius = np.mean(np.linalg.norm(u_hat.atoms[0].support.boundary_vertices, axis=1))

print("radius: {}".format(radius))
print("weight: {}".format(weight))